In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras

/Users/PeterHayu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
X_train =  pd.read_csv('X_train.csv')
Y_train = pd.read_csv('y_train.csv')
X_test =  pd.read_csv('X_test.csv')
Y_test = pd.read_csv('y_test.csv')

In [4]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

ValueError: could not convert string to float: 'Kerry: ISIS  is on the run  despite recent terror attacks'

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
howManySelectedFeaturesPutIntoModel = 20

In [221]:
#unit = number of neurons in the layer
clf1 = Sequential([
    Dense(units=9, kernel_initializer='normal', input_dim=howManySelectedFeaturesPutIntoModel, activation='relu'),
    Dense(units=14, kernel_initializer='normal', activation='softmax'),
    Dropout(0.25),
    Dense(1, kernel_initializer='normal', activation='sigmoid')
])

In [224]:
clf1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_87 (Dense)             (None, 9)                 189       
_________________________________________________________________
dropout_26 (Dropout)         (None, 9)                 0         
_________________________________________________________________
dense_88 (Dense)             (None, 1)                 10        
Total params: 199
Trainable params: 199
Non-trainable params: 0
_________________________________________________________________


In [94]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping


# define roc_callback, inspired by https://github.com/keras-team/keras/issues/6050#issuecomment-329996505
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [222]:
#metrics can use 'Accuracy'
clf1.compile(optimizer='adam', loss='binary_crossentropy', metrics=[auc_roc])

In [223]:
clf1.fit(X_train, Y_train, batch_size=15, epochs=2)

Epoch 1/2
668982/668982 [==============================] - 78s 116us/step - loss: 0.0558 - auc_roc: 0.7381
Epoch 2/2
668982/668982 [==============================] - 76s 114us/step - loss: 0.0494 - auc_roc: 0.7502


In [ ]:
score = clf.evaluate(X_test, Y_test, batch_size=128)
print('\nAnd the Score is ', score[1] * 100, '%')